summarizer 설치

In [ ]:
!pip install bert-extractive-summarizer
!pip install spacy==2.1.3
!pip install transformers==2.2.2
!pip install neuralcoref

!python -m spacy download en_core_web_md

한글 <-> 로마자 변환 함수

In [ ]:
class Cstr(str):
    ord_upper = ord('天')
    ord_lower = ord('地')
    ord_const_upper = ord_upper - ord('A')
    ord_const_lower = ord_lower - ord('a')
    def __init__(self, obj):
        super().__init__()
        self.content = obj
    
    def to_specialChr(self):
        for idx, c in enumerate(self.content):
            if ord(c) >= ord('A') and ord(c) <= ord('Z'):
                self.content = (self.content[:idx] + 
                                chr(ord(c) + self.ord_const_upper) + 
                                self.content[idx+1:])
            elif ord(c) >= ord('a') and ord(c) <= ord('z'):
                self.content = (self.content[:idx] + 
                                chr(ord(c) + self.ord_const_lower) + 
                                self.content[idx+1:])
        return Cstr(self.content)

    def to_originChr(self):
        #print('to_originChr')
        for idx, c in enumerate(self.content):
            if ord(c) >= self.ord_upper and ord(c) <= (self.ord_upper +25):
                self.content = (self.content[:idx] + 
                                chr(ord(c) - self.ord_const_upper) + 
                                self.content[idx+1:])
            elif ord(c) >= self.ord_lower and ord(c) <= (self.ord_lower +25):
                self.content = (self.content[:idx] + 
                                chr(ord(c) - self.ord_const_lower) + 
                                self.content[idx+1:])
        return Cstr(self.content)

In [ ]:
# from cstr import Cstr
# test=Cstr('hi')
# print(test)
# print(type(test))

# >>> import krt
# hi
# <class 'krt.utils.cstr.Cstr'>

def hangulize(text, toEncoding):
  """
    Take romanized Korean text and return
    the hangulized version.
  """
  output = ''
  toLoop = []

  """
    Cstr class is added to make this module case sensitive
  """
  if type(text) != Cstr:
    text = Cstr(text)
                                # from special chr to normal chr

  for i in text:
    toLoop.append(i)
  toLoop.reverse()
  # print toLoop
  
  # Set up the vars for the state machine
  state = 'START' # possible states: START, INITIAL, VOWEL, FINAL
  syllable = {}
  hangul = ''
  
  # Start the state machine
  while (len(toLoop) > 0):
    
    # get the next char
    char = toLoop.pop()
    if state == 'START':
      syllable = {}
      # check to make sure it's a letter, if not, just add it to the string
      if char in letters:
        if char in consonantLetters:
          syllable['initial'] = char
          state = 'INITIAL'
          if len(toLoop) == 0:
            if char in nonpachim:
              hangul += hangulize_syllable(syllable)
            else:
              hangul += syllable['initial']
        elif char in vowelLetters:
          syllable['vowel'] = char
          state = 'VOWEL'
          if len(toLoop) == 0:
            if char in moum:
              hangul += hangulize_syllable(syllable)
            else:
              hangul += char
      else:
        if char == '.' and len(toLoop) > 0 and toLoop[-1] in letters:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
            char = ''
          else:
            if char != '.':
              hangul += char
              char = ''
          state = 'START'
        elif char == '<':
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          state = 'SINGLE'
        else:
          # print char + ' not in letters, not a period, and not a lesser than sign'
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          hangul += char
          char = ''
          state = 'START'
    elif state == 'INITIAL':
      if char in letters:
        if char in vowelLetters:
          # todo: but what if this char is 'w' and the next one is a consonant?
          syllable['vowel'] = char
          state = 'VOWEL'
          if char in moum:
            fullVowel = True
        else:
          if syllable['initial'] + char in nonpachim or syllable['initial'] + char in consonantLetters:
            syllable['initial'] = syllable['initial'] + char
          else:
            hangul += hangulize_syllable(syllable)
            syllable = {}
            state = 'START'
      else:
        if char == '.' and len(toLoop) > 0 and toLoop[-1] in letters:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
            char = ''
          else:
            if char != '.':
              hangul += char
              char = ''
          state = 'START'
        elif char == '<':
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          state = 'SINGLE'
        else:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          hangul += char
          char = ''
          state = 'START'
    elif state == 'VOWEL':
      #print 'state is VOWEL'
      if char in letters:
        if char in vowelLetters:
          if syllable['vowel'] + char in moum:
            syllable['vowel'] = syllable['vowel'] + char
          else:
            print(type(syllable), syllable)
            hangul += hangulize_syllable(syllable)
            syllable = {}
            syllable['initial'] = ''
            syllable['vowel'] = char
            state = 'VOWEL'
        else:
          syllable['final'] = char
          state = 'FINAL'
      else:
        if char == '.' and len(toLoop) > 0 and toLoop[-1] in letters:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
            char = ''
          else:
            if char != '.':
              hangul += char
              char = ''
          state = 'START'
        elif char == '<':
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          state = 'SINGLE'
        else:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          hangul += char
          char = ''
          state = 'START'
    elif state == 'FINAL':
      if char in letters:
        if syllable['final'] + char in pachim:
          syllable['final'] = syllable['final'] + char
        else:
          hangul += hangulize_syllable(syllable)
          syllable = {}
          if char in vowelLetters:
            syllable['initial'] = ''
            syllable['vowel'] = char
            state = 'VOWEL'
            if char in moum:
              fullVowel = True
          else:
            syllable['initial'] = char
            state = 'INITIAL'
      else:
        hangul += hangulize_syllable(syllable)
        syllable = {}
        if char == '.' and len(toLoop) > 0 and toLoop[-1] in letters:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
            char = ''
          else:
            if char != '.':
              hangul += char
              char = ''
          state = 'START'
        elif char == '<':
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          state = 'SINGLE'
        else:
          if syllable != {}:
            hangul += hangulize_syllable(syllable)
            syllable = {}
          hangul += char
          char = ''
          state = 'START'
    elif state == 'SINGLE':
      # print 'in state == SINGLE: ',
      # print str(syllable),
      # print "CURRENT LETTER: " + char
      if char in singleLetters:
        # print 'char in singleLetters'
        if 'single' in syllable and char + syllable['single'] in singleLetters:
          # print syllable['single'] + ' in singleLetters'
          syllable['single'] += char
        elif 'single' in syllable and not syllable['single'] + char in singleLetters:
          # print syllable['single'] + char + ' is not in singleLetters'
          hangul += '<' + syllable['single']
          syllable = {}
          if char in vowelLetters:
            syllable['initial'] = ''
            syllable['vowel'] = char
            state = 'VOWEL'
          elif char in consonantLetters:
            syllable['initial'] = char
            state = 'INITIAL'
          else:
            state = 'START'
        else:
          if 'single' in syllable:
            syllable['single'] += char
          else:
            syllable['single'] = char
      elif char == '>':
        # print 'ending single',
        hangul += hangulize_syllable(syllable)
        syllable = {}
        char = ''
        state = 'START'
      else:
        # print char + ' not in singleLetters'
        hangul += char
        syllable = {}
        state = 'START'
    if len(toLoop) == 0:
      if syllable == {}:
        hangul += char
      else:
        if 'single' in syllable:
          hangul += '<' + syllable['single']
        else:
          hangul += hangulize_syllable(syllable)
    continue
#  return Cstr(hangul#.encode(toEncoding))
#  return Cstr(hangul.encode(toEncoding))
  return Cstr(hangul).to_originChr()#.encode(toEncoding)


def hangulize_syllable(syl):
  """
    Take romanized Korean word and return a list of its syllables.
  """
  single = vowel = final = None
  initial = ''
  # print str(syl)
  if 'single' in syl:
    single = syl['single']
  if 'initial' in syl:
    initial = syl['initial']
  if 'vowel' in syl:
    vowel = syl['vowel']
  if 'final' in syl:
    final = syl['final']
  #print initial
  #print vowel
  #print final
  if single == None:
    if vowel == None and final == None:
      #print "is single initial"
      single = initial
    elif initial == None and final == None:
      #print 'is single vowel'
      single = vowel
    else:
      #print 'is not single'
      single = None
  
  sIndex = iIndex = vIndex = fIndex = 0
  if single != None:
    if single in singlesDict:
      sIndex = singlesDict[single]
    else:
      return single
    #print 'getting singles index.... ' + str(sIndex)
  else:
    if initial in nonpachimDict:
      iIndex = nonpachimDict[initial]
    else:
      return initial + vowel + final
    if vowel in moumDict:
      vIndex = moumDict[vowel]
    else:
      return initial + vowel + final
    if final in pachimDict:
      fIndex = pachimDict[final]
    else:
      fIndex = None
    sIndex = None
    #print iIndex
    #print vIndex
    #print fIndex
  
  if sIndex != None:
    return chr(sIndex + 12593)
  else:
    total = 44032
    if iIndex != None:
      total += iIndex * 588
    if vIndex != None:
      total += 28 * vIndex
    if fIndex != None:
      total += fIndex
#     return chr(total)
    # python2 chr() >>> python3 chr()
    return chr(total)
  return
    
      
    
def romanize(raw, fromEnc = 'utf8', toEnc = 'utf8'):
  """
    Takes a raw string of Korean, a 'from encoding' and a 'to encoding'.
    Returns a romanized string of the text, encoded as specified (default
    'from encoding' is None and default 'to encoding' is utf-8).
  """
#   if fromEnc != None:
#     raw = raw.decode(fromEnc)
  """
    Cstr class is added to make this module case sensitive
  """
  raw = Cstr(raw).to_specialChr()    # to convert 
                                     # from normal chr to special chr
  newString = ''

  for i in range(len(raw)):
    index = gti(raw[i])
    
    # If the index is a single (non-syllabic) hangul letter
    if index in range(12593, 12687):
      index = index - 12593
      if singles[index] and len(newString) > 1 and newString[-1] != ' ':
        newString += '.'
      newString += '<' + singles[index] + '>'
   
    # If the index represents a hangul syllable
    elif index in range(44032, 55204):
      index = index - 44032
      # ! int형으로 변환
      initial = int(index / 588)
      vowel = int((index % 588) / 28)
      final = int((index % 588) % 28)
      if len(newString) > 0:
        if nonpachim[initial] == 'g' and newString[-1] == 'n':
          newString += '.'
        elif nonpachim[initial] == '' and newString[-2:len(newString)] == 'ng':
          newString += '.'
        elif (newString[-1] in moum or newString[-2:len(newString)] in moum) and nonpachim[initial] in pachim + nonpachim:
          newString += '.'
        elif nonpachim[initial] == '' and newString[-1] in pachim + nonpachim: 
          newString += '.'
        elif nonpachim[initial] == 'h' and newString[-1] in ['t','k','p','c','n','l']: 
          newString += '.'
        elif newString[-1] + nonpachim[initial] in pachim + nonpachim or (len(nonpachim[initial]) > 1 and newString[-1] + nonpachim[initial][0] in pachim + nonpachim):
          newString += '.' 

      newString += nonpachim[initial]
      newString += moum[vowel]
      newString += pachim[final]
    
    # Otherwise
    else:
      newString += chr(index).upper()
#  return Cstr(newString.encode(toEnc))
  return Cstr(newString)#.encode(toEnc)


def gti(char):
  """
    Only accepts unicode characters
    Return index of characters
  """
  return ord(char)


# Character lists
singles = ['k', 'kk', 'ks', 'n', 'nc', 'nh', 't', 'tt', 'l', 'lk', 'lm', 'lp', 'ls', 'lth', 'lph', 'lh', 'm', 'p', 'pp', 'ps', 's', 'ss', 'ng', 'c', 'cc', 'ch', 'kh', 'th', 'ph', 'h', 'a', 'ay', 'ya', 'yay', 'e', 'ey', 'ye', 'yey', 'o', 'wa', 'way', 'oy', 'yo', 'wu', 'we', 'wey', 'wi', 'yu', 'u', 'uy', 'i', 'NONE', 'NN', 'NT', 'NS', 'NZ', 'LKS', 'LT', 'LPS', 'LZ', 'LH', 'MP', 'MS', 'MZ', 'MNG', 'PK', 'PT', 'PSK', 'PST', 'PC', 'PTH', 'PNG', 'PPNG', 'SK', 'SL', 'ST', 'SP', 'SC', 'Z', 'NGNG', 'NG', 'NGS', 'NGZ', 'PHNG', 'HH', 'H', 'YOYA', 'YOYAY', 'YOI', 'YUE', 'YUEY', 'YUI', 'A', 'E']
moum = ['a', 'ay', 'ya', 'yay', 'e', 'ey', 'ye', 'yey', 'o', 'wa', 'way', 'oy', 'yo', 'wu', 'we', 'wey', 'wi', 'yu', 'u', 'uy', 'i']
pachim = ['', 'k', 'kk', 'ks', 'n', 'nc', 'nh', 't', 'l', 'lk', 'lm', 'lp', 'ls', 'lth', 'lph', 'lh', 'm', 'p', 'ps', 's', 'ss', 'ng', 'c', 'ch', 'kh', 'th', 'ph', 'h']
nonpachim = ['k', 'kk', 'n', 't', 'tt', 'l', 'm', 'p', 'pp', 's', 'ss', '', 'c', 'cc', 'ch', 'kh', 'th', 'ph', 'h']
letters = ['k', 'n', 't', 'l', 'm', 'p', 's', 'c', 'k', 'h', 'g', 'a', 'y', 'e', 'o', 'w', 'u', 'i']
vowelLetters = ['a', 'e', 'i', 'o', 'u', 'w', 'y']
consonantLetters = ['c', 'g', 'h', 'k', 'l', 'm', 'n', 'p', 's', 't', 'lt', 'lp']
singleLetters = ['k', 'kk', 'ks', 'ns', 'n', 'nc', 'nh', 't', 'tt', 'l', 'lk', 'lm', 'lp', 'ls', 'lth', 'lt', 'lph', 'lt', 'lh', 'm', 'p', 'pp', 'ps', 's', 'ss', 'ng', 'c', 'cc', 'ch', 'kh', 'th', 'ph', 'h', 'a', 'ay', 'ya', 'yay', 'e', 'ey', 'ye', 'yey', 'o', 'wa', 'way', 'oy', 'yo', 'wu', 'we', 'wey', 'wi', 'yu', 'u', 'uy', 'i', 'NONE', 'N', 'NO', 'NON', 'NN', 'NT', 'NS', 'NZ', 'LKS', 'LK', 'L', 'LT', 'LPS', 'LP', 'LZ', 'LH', 'MP', 'M', 'MS', 'MZ', 'MNG', 'MN', 'PK', 'P', 'PT', 'PSK', 'PS', 'PST', 'PC', 'PTH', 'PNG', 'PPNG', 'PP', 'PPN', 'SK', 'S', 'SL', 'ST', 'SP', 'SC', 'Z', 'NGNG', 'NGN', 'NG', 'NGS', 'NGZ', 'PHNG', 'PH', 'PHN', 'HH', 'H', 'YOYA', 'Y', 'YO', 'O', 'YOY', 'YOYAY', 'YOI', 'YUE', 'YU', 'YUEY', 'YUI', 'A', 'E']


# Character dictionaries
singlesDict = {'yey': 37, 'YUE': 89, 'YOYAY': 87, 'tt': 7, 'lm': 10, 'lk': 9, 'lh': 15, 'ls': 12, 'lp': 11, 'wey': 45, 'YUEY': 90, 'yo': 42, 'ya': 32, 'H': 85, 'LPS': 58, 'yu': 47, 'YUI': 91, 'h': 29, 'l': 8, 'p': 17, 't': 6, 'HH': 84, 'ey': 35, 'NGS': 81, 'n': 3, 'LKS': 56, 'NGZ': 82, 'NONE': 51, 'PT': 66, 'PTH': 70, 'PC': 69, 'PK': 65, 'MNG': 64, 'we': 44, 'wa': 39, 'PPNG': 72, 'wi': 46, 'wu': 43, 'PSK': 67, 'c': 23, 'k': 0, 'o': 38, 'PHNG': 83, 's': 20, 'MP': 61, 'MS': 62, 'YOYA': 86, 'lth': 13, 'PST': 68, 'MZ': 63, 'ch': 25, 'cc': 24, 'ps': 19, 'pp': 18, 'yay': 33, 'NN': 52, 'NG': 80, 'NZ': 55, 'way': 40, 'ph': 28, 'NS': 54, 'NT': 53, 'th': 27, 'Z': 78, 'uy': 49, 'SP': 76, 'ST': 75, 'SK': 73, 'ss': 21, 'SL': 74, 'SC': 77, 'ay': 31, 'NGNG': 79, 'nh': 5, 'nc': 4, 'LH': 60, 'ng': 22, 'LT': 57, 'ks': 2, 'LZ': 59, 'A': 92, 'E': 93, 'oy': 41, 'YOI': 88, 'ye': 36, 'kk': 1, 'a': 30, 'e': 34, 'i': 50, 'kh': 26, 'm': 16, 'u': 48, 'lph': 14, 'PNG': 71}
moumDict = {'a': 0, 'we': 14, 'uy': 19, 'yay': 3, 'oy': 11, 'wa': 9, 'ya': 2, 'yo': 12, 'ye': 6, 'o': 8, 'yey': 7, 'i': 20, 'wu': 13, 'ey': 5, 'wi': 16, 'way': 10, 'ay': 1, 'e': 4, 'wey': 15, 'yu': 17, 'u': 18}
pachimDict = {'': 0, 'nc': 5, 'ch': 23, 'nh': 6, 'ps': 18, 'p': 17, 'lm': 10, 'lk': 9, 'lh': 15, 'ng': 21, 'ls': 12, 'lp': 11, 'ph': 26, 'th': 25, 'c': 22, 'ss': 20, 'h': 27, 'k': 1, 'kh': 24, 'm': 16, 'l': 8, 'n': 4, 'ks': 3, 'kk': 2, 's': 19, 't': 7, 'lph': 14, 'lth': 13}
nonpachimDict = {'': 11, 'pp': 8, 'ch': 14, 'ss': 10, 'kk': 1, 'c': 12, 'k': 0, 'kh': 15, 'm': 6, 'l': 5, 'n': 2, 'p': 7, 's': 9, 't': 3, 'th': 16, 'h': 18, 'ph': 17, 'tt': 4, 'cc': 13}


In [ ]:
print(hangulize('swuyengcangeyse hwunlyen halttay yenghwa cciknun cangmyen pwasnuntey cengmal yelsimhi yenghwa chwalyengul hasitelakwuyo ~~ sinmina enni silceylo swuyengcangeyse pwassnuntey cengmalyeyppeyo<wu><wu><wu>chwalyeng swukohasiko himnayseyyong!!', 'utf8'))

<class 'dict'> {'initial': 's', 'vowel': 'wu'}
<class 'dict'> {'initial': '', 'vowel': 'wu'}
<class 'dict'> {'initial': 's', 'vowel': 'wu'}
<class 'dict'> {'initial': 'p', 'vowel': 'ey'}
<class 'dict'> {'initial': '', 'vowel': 'ey'}
수영장엣어 훈련 할때 영화 찍는 장면 봣는데 정말 엸임히 영화 촬영을 핫읻얼악우요 ~~ 신민아 언니 실젤오 수영장엣어 봤는데 정말옙베오w으>w으>w으>촬영 숙옿앗익오 힘냇에용!!


1) body : 한글 그대로

2) rbody : romanize 거쳐서 

In [ ]:
from google.colab import drive
import os
import pandas as pd
import torch

drive.mount('/content/gdrive')
path = '/content/gdrive/Shared drives/16통계/data/'

movies = pd.read_csv(path + "navercombine.csv")
print(movies.columns)
body = movies.review
print(body[0:5])

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wH0uVpjSvHf7dVMF-LQrZ3UZi8wWYe0wCXEWCOW8gsDj1wow3nYl0Q
Mounted at /content/gdrive
Index(['Unnamed: 0', 'title', 'director', 'country', 'ganre', 'info', 'number',
       'review'],
      dtype='object')
0      반지하라도 ‘우리’집이 있었다. 이제는 ‘우리’집이 아닌 할아버지 댁으로 들어가...
1    부산국제영화제에서 처음 봤던 영화는 '너의 이름은' 이라는 애니였다. 친구가 표를 ...
2    영화 기기괴괴 성형수를\r\n시사회로 보고왔어요\r\n\r\n 영화 기기괴괴 성형수...
3    나이브스 아웃 범인은 우리안에 있다 추리 스릴러\r\n\r\n나이브스 아웃, 줄거리...
4    먼 훗날 우리\r\n감독\r\n유약영\r\n출연\r\n정백연, 주동우, 톈좡좡\r\...
Name: review, dtype: 

In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
if torch.cuda.is_available():
  print('o')
else:
  print('x')

o


summarizer options

model = Summarizer(
  
    model: This gets used by the hugging face bert library to load the model, you can supply a custom trained model here

    custom_model: If you have a pre-trained model, you can add the model class here.

    custom_tokenizer:  If you have a custom tokenizer, you can add the tokenizer here.

    hidden: Needs to be negative, but allows you to pick which layer you want the embeddings to come from.

    reduce_option: It can be 'mean', 'median', or 'max'. This reduces the embedding layer for pooling.

    sentence_handler: The handler to process sentences. If want to use coreference, instantiate and pass CoreferenceHandler instance
)

model(

    body: str # The string body that you want to summarize

    ratio: float # The ratio of sentences that you want for the final summary

    min_length: int # Parameter to specify to remove sentences that are less than 40 characters

    max_length: int # Parameter to specify to remove sentences greater than the max length,

    num_sentences: Number of sentences to use. Overrides ratio if supplied.
)

In [ ]:
summ = []
# body = movies.review
for i in range(0,len(body)):
  print(i)
  rbody = body[i]
  rbody = romanize(rbody)
  rbody = rbody.replace('.','')
  rbody = rbody.replace('\r','')
  rbody = rbody.replace('\n','')
  rbody = rbody.replace('[','')
  rbody = rbody.replace(']','')
  rbody = rbody.replace('%','')
  rbody = rbody.replace('☆','')
  rbody = rbody.replace('★','')
  result= model(rbody,min_length=0, max_length=900)
  full = ''.join(result)
  summ.append(full)

movies['summary'] = summ

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


In [ ]:
os.chdir(path)
print(os.getcwd())
movies.to_csv("mSumm.csv",index=False)

/content/gdrive/Shared drives/16통계/data


In [ ]:
# !ls
os.chdir(path)
data = pd.read_csv("mSumm.csv")
print(data.head())

   Unnamed: 0  ...                                            summary
0           0  ...  ayekpaywutuluy eselphun tus kasumul wullinun k...
1           1  ...  inun kwake cipuli aynieyse sahoycek mwunceyey ...
2           2  ...  han ttay palleylinayessten yeycinun 坆坆坆圼坈坂坃地坁圽圴场坁
3           3  ...  kyengchalun manyakul taypihay swusalul sicakha...
4           5  ...  -kuttay neyka yongki nayse cihacheley ollathas...

[5 rows x 9 columns]


In [ ]:
print(len(data))
naIdx = data[data['summary'].isnull()].index
print(len(naIdx))
data = data.drop(naIdx)
print(len(data))
data.to_csv("mSumm.csv",index=False)

97
0
97


In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 6.5MB/s 


In [ ]:
print(data.columns)

Index(['Unnamed: 0', 'title', 'director', 'country', 'ganre', 'info', 'number',
       'review', 'summary'],
      dtype='object')


In [ ]:
writer = pd.ExcelWriter('mSumm.xlsx', options={'strings_to_urls': False}) #엑셀 255 글자 수 제한 해결
data = data.drop(['review','Unnamed: 0'], axis=1)
data = data[['number','title','director','ganre','country','info','summary']]
print(data.columns)
data.to_excel(writer, index=False , engine='xlsxwriter')
writer.close()

Index(['number', 'title', 'director', 'ganre', 'country', 'info', 'summary'], dtype='object')


In [ ]:
# drive.mount('/content/drive/')
# print(os.chdir('/content/drive/My Drive/Colab Notebooks/'))
!ls

booksSumm.csv  bookSumm.xlsx   mSumm.csv   navercombine.csv  yes24.csv
bookSumm.csv   moviesSumm.csv  mSumm.xlsx  totalbookimg      yes24Img


In [ ]:
temp = pd.read_excel('mSumm.xlsx')
print(temp.columns)

Index(['number', 'title', 'director', 'ganre', 'country', 'info', 'summary'], dtype='object')
